In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchsummary import summary

import time
import os
import numpy as np
import argparse
import datetime
import itertools
import seaborn as sns
from sklearn import metrics
from matplotlib import pyplot as plt

import modules
from utils import dataset
from modules import model
from imp import reload

In [ ]:
device = torch.device('cuda')

In [ ]:
normalize = torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256,256)),
                                torchvision.transforms.RandomHorizontalFlip(),
                                torchvision.transforms.RandomVerticalFlip(),
                                torchvision.transforms.RandomCrop(220),
                                torchvision.transforms.ToTensor(),
                                normalize])
norm_transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), normalize])
test_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256,256)),
                                torchvision.transforms.RandomCrop(220),
                                torchvision.transforms.ToTensor(),
                                normalize])

In [ ]:
training_data = dataset.ImageFolderPaths('./data/Leukemia/train', transform = transform)
test_data = dataset.ImageFolderPaths('./data/Leukemia/test/', transform = test_transform)

In [ ]:
data_loader = torch.utils.data.DataLoader(training_data,
                                          batch_size= 16,
                                          shuffle=True,
                                          num_workers= 0)

test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size= 1,
                                          shuffle=True,
                                          num_workers= 1)

In [ ]:
learning_rate = 1e-4
beta1 = 0.5
beta2 = 0.999
num_epochs = 1000
latent_size = 200

In [ ]:
dis_BCElogit_criterion = nn.BCEWithLogitsLoss()
dis_criterion = nn.CrossEntropyLoss()
aen_criterion = nn.MSELoss()

In [ ]:
enc = model.Encoder_Leukemia(latent_size)
gen = model.Generator_Leukemia(latent_size)
dis_xz = model.Discriminator_xz_Leukemia(latent_size, 0.2)
dis_xx = model.Discriminator_xx_Leukemia(latent_size, 0.2)
dis_zz = model.Discriminator_zz_Leukemia(latent_size, 0.2)

In [ ]:
acc_lam = {}
auroc = {}

# random seed 0

In [ ]:
gen.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/0/Leukemia1000epochs2020-03-30-13-40-38G'))
enc.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/0/Leukemia1000epochs2020-03-30-13-40-38E'))
dis_xz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/0/Leukemia1000epochs2020-03-30-13-40-38D_xz'))
dis_xx.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/0/Leukemia1000epochs2020-03-30-13-40-38D_xx'))
dis_zz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/0/Leukemia1000epochs2020-03-30-13-40-38D_zz'))

In [ ]:
dis_xz.to(device)
dis_xx.to(device)
dis_zz.to(device)
enc.to(device)
gen.to(device);

In [ ]:
dis_xz.eval()
dis_xx.eval()
dis_zz.eval()
enc.eval()
gen.eval()

loss_neg = torch.zeros((5,1)).cuda()
loss_pos = torch.zeros((15,1)).cuda()
c_neg = c_pos = 0
c_neg_val = c_pos_val = cntr = 0
for step, (images, labels, path) in enumerate(test_loader, 0):
    if '0.tif' in str(path):
        if c_neg_val < 5:
            images = images.view(-1, 3, 220, 220)
            c_neg_val += 1
            cntr += 1
        else:
            break
    elif '1.tif' in str(path):
        if c_pos_val < 15:
            images = images.view(-1, 3, 220, 220)
            c_pos_val += 1
            cntr += 1
        else:
            break
    
    x_real_test = images.cuda()
    z_random = torch.randn(images.shape[0], latent_size, 1, 1).cuda()
    z_gen = enc(x_real_test)
    x_gen = gen(z_random)
    rec_x = gen(z_gen)
    rec_z = enc(x_gen)
    
    l_gen, _ = dis_xz(x_real_test, z_gen)
    l_enc, _ = dis_xz(x_gen, z_random)
    
    x_logit_real, inter_layer_inp = dis_xx(x_real_test, x_real_test)
    x_logit_fake, inter_layer_rct = dis_xx(x_real_test, rec_x)
   
    fm = inter_layer_inp - inter_layer_rct
    feature_loss = torch.norm(fm, 1, keepdim=False) 
    feature_loss = feature_loss.squeeze()
    
    if '0.tif' in str(path):
        loss_neg[c_neg] = feature_loss.detach()
        c_neg += 1
    else:
        loss_pos[c_pos] = feature_loss.detach()
        c_pos += 1

    if cntr == 20:
        break
mean_neg = torch.mean(loss_neg)
mean_pos = torch.mean(loss_pos)
total_mean = (mean_neg + mean_pos) / 2
print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))
print ('threshold: %0.4f' %(total_mean))

In [ ]:
lam = 0.1
seed = 0
loss_neg = torch.zeros((30,1)).cuda()
loss_pos = torch.zeros((130,1)).cuda()
c_loss = c_neg = c_pos = 0
for step, (images, labels, path) in enumerate(test_loader, 0):
    images = images.view(-1, 3, 220, 220)
    dis_xz.eval()
    dis_xx.eval()
    dis_zz.eval()
    enc.eval()
    gen.eval()
    
    x_real_test = images.cuda()
    z_random = torch.randn(images.shape[0], latent_size, 1, 1).cuda()
    z_gen = enc(x_real_test)
    x_gen = gen(z_random)
    rec_x = gen(z_gen)
    rec_z = enc(x_gen)
    
    l_gen, _ = dis_xz(x_real_test, z_gen)
    l_enc, _ = dis_xz(x_gen, z_random)
    
    x_logit_real, inter_layer_inp = dis_xx(x_real_test, x_real_test)
    x_logit_fake, inter_layer_rct = dis_xx(x_real_test, rec_x)
   
    fm = inter_layer_inp - inter_layer_rct
    feature_loss = torch.norm(fm, 1, keepdim=False) 
    feature_loss = feature_loss.squeeze()
    
    if '0.tif' in str(path):
        loss_neg[c_neg] = feature_loss.detach()
        c_neg += 1
    else:
        loss_pos[c_pos] = feature_loss.detach()
        c_pos += 1
print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))

In [ ]:
x1 = loss_neg.cpu().numpy()
x2 = loss_pos.cpu().numpy()
sns.distplot(x1, hist=False, kde=True, kde_kws={'linewidth': 3}, label='Normal')
sns.distplot(x2, hist=False, kde=True, kde_kws={'linewidth': 3}, label='Anomalous')
plt.title('Distribution of normal and abnormal samples')
plt.xlabel('Anomaly Score');

FP = TP = []
neg_pre_wrong = 0
for i in range(len(loss_neg)):
    if loss_neg[i] > 500:
        neg_pre_wrong += 1

pos_pre_wrong = 0
for i in range(len(loss_pos)):
    if loss_pos[i] <= 500:
        pos_pre_wrong += 1
print ("number of normal samples missclassified: %d, number of anomalous samples missclassified: %d" 
       %(neg_pre_wrong, pos_pre_wrong))
tp = (len(loss_pos) - pos_pre_wrong)
fn = pos_pre_wrong
fp = neg_pre_wrong
tn = len(loss_neg) - neg_pre_wrong
precision = tp / (tp + fp)
## recall / sensitivity / True Positive Rate
recall = tp / (tp + fn)
## False Positive Rate / 1 - Specificity
fp_rate = fp / (fp + tn)
specificity = tn / (tn + fp)
f1 = 2 * ((precision * recall)/(precision + recall))
accuracy = (tp + tn) / (tp + tn + fp + fn)
acc_lam[seed] = accuracy
print ("tp: %d, fp: %d, fn: %d, tn: %d" %(tp, fp, fn, tn))
print ("precision: %.5f, recall: %.5f, specificity: %.5f, f1: %.5f, fp_rate: %.5f, accuracy: %.5f" 
       %(precision, recall, specificity, f1, fp_rate, accuracy))
anomalous = torch.ones((len(loss_pos), 1))
normal = torch.zeros((len(loss_neg), 1))
y = torch.cat((anomalous, normal), 0)
scores = torch.cat((loss_pos, loss_neg), 0)
fpr, tpr, thresholds = metrics.roc_curve(y.cpu(), scores.cpu())
auc = metrics.auc(fpr, tpr)
auroc[seed] = auc
print ('AUC', auc)

# random seed 10

In [ ]:
gen.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/10/Leukemia1000epochs2020-03-15-17-59-22G'))
enc.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/10/Leukemia1000epochs2020-03-15-17-59-22E'))
dis_xz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/10/Leukemia1000epochs2020-03-15-17-59-22D_xz'))
dis_xx.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/10/Leukemia1000epochs2020-03-15-17-59-22D_xx'))
dis_zz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/10/Leukemia1000epochs2020-03-15-17-59-22D_zz'))

In [ ]:
dis_xz.to(device)
dis_xx.to(device)
dis_zz.to(device)
enc.to(device)
gen.to(device);

In [ ]:
dis_xz.eval()
dis_xx.eval()
dis_zz.eval()
enc.eval()
gen.eval()

loss_neg = torch.zeros((5,1)).cuda()
loss_pos = torch.zeros((15,1)).cuda()
c_neg = c_pos = 0
c_neg_val = c_pos_val = cntr = 0
for step, (images, labels, path) in enumerate(test_loader, 0):
    if '0.tif' in str(path):
        if c_neg_val < 5:
            images = images.view(-1, 3, 220, 220)
            c_neg_val += 1
            cntr += 1
        else:
            break
    elif '1.tif' in str(path):
        if c_pos_val < 15:
            images = images.view(-1, 3, 220, 220)
            c_pos_val += 1
            cntr += 1
        else:
            break
    
    x_real_test = images.cuda()
    z_random = torch.randn(images.shape[0], latent_size, 1, 1).cuda()
    z_gen = enc(x_real_test)
    x_gen = gen(z_random)
    rec_x = gen(z_gen)
    rec_z = enc(x_gen)
    
    l_gen, _ = dis_xz(x_real_test, z_gen)
    l_enc, _ = dis_xz(x_gen, z_random)
    
    x_logit_real, inter_layer_inp = dis_xx(x_real_test, x_real_test)
    x_logit_fake, inter_layer_rct = dis_xx(x_real_test, rec_x)
   
    fm = inter_layer_inp - inter_layer_rct
    feature_loss = torch.norm(fm, 1, keepdim=False) 
    feature_loss = feature_loss.squeeze()
    
    if '0.tif' in str(path):
        loss_neg[c_neg] = feature_loss.detach()
        c_neg += 1
    else:
        loss_pos[c_pos] = feature_loss.detach()
        c_pos += 1

    if cntr == 20:
        break
mean_neg = torch.mean(loss_neg)
mean_pos = torch.mean(loss_pos)
total_mean = (mean_neg + mean_pos) / 2
print ('model 1')
print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))
print ('threshold: %0.4f' %(total_mean))

In [ ]:
lam = 0.1
seed = 10
loss_neg = torch.zeros((100,1)).cuda()
loss_pos = torch.zeros((130,1)).cuda()
c_loss = c_neg = c_pos = 0
for step, (images, labels, path) in enumerate(test_loader, 0):
    images = images.view(-1, 3, 220, 220)
    dis_xz.eval()
    dis_xx.eval()
    dis_zz.eval()
    enc.eval()
    gen.eval()
    
    x_real_test = images.cuda()
    z_random = torch.randn(images.shape[0], latent_size, 1, 1).cuda()
    z_gen = enc(x_real_test)
    x_gen = gen(z_random)
    rec_x = gen(z_gen)
    rec_z = enc(x_gen)
    
    l_gen, _ = dis_xz(x_real_test, z_gen)
    l_enc, _ = dis_xz(x_gen, z_random)
    
    x_logit_real, inter_layer_inp = dis_xx(x_real_test, x_real_test)
    x_logit_fake, inter_layer_rct = dis_xx(x_real_test, rec_x)
   
    fm = inter_layer_inp - inter_layer_rct
    feature_loss = torch.norm(fm, 1, keepdim=False) 
    feature_loss = feature_loss.squeeze()
    
    if '0.tif' in str(path):
        loss_neg[c_neg] = feature_loss.detach()
        c_neg += 1
        c_loss += 1
    else:
        loss_pos[c_pos] = feature_loss.detach()
        c_pos += 1
        c_loss += 1
print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))

In [ ]:
FP = TP = []
neg_pre_wrong = 0
for i in range(len(loss_neg)):
    if loss_neg[i] > 1100:
        neg_pre_wrong += 1

pos_pre_wrong = 0
for i in range(len(loss_pos)):
    if loss_pos[i] <= 1100:
        pos_pre_wrong += 1
print ("number of normal samples missclassified: %d, number of anomalous samples missclassified: %d" 
       %(neg_pre_wrong, pos_pre_wrong))
tp = (len(loss_pos) - pos_pre_wrong)
fn = pos_pre_wrong
fp = neg_pre_wrong
tn = len(loss_neg) - neg_pre_wrong
precision = tp / (tp + fp)
## recall / sensitivity / True Positive Rate
recall = tp / (tp + fn)
## False Positive Rate / 1 - Specificity
fp_rate = fp / (fp + tn)
specificity = tn / (tn + fp)
f1 = 2 * ((precision * recall)/(precision + recall))
accuracy = (tp + tn) / (tp + tn + fp + fn)
acc_lam[seed] = accuracy
print ("tp: %d, fp: %d, fn: %d, tn: %d" %(tp, fp, fn, tn))
print ("precision: %.5f, recall: %.5f, specificity: %.5f, f1: %.5f, fp_rate: %.5f, accuracy: %.5f" 
       %(precision, recall, specificity, f1, fp_rate, accuracy))
anomalous = torch.ones((len(loss_pos), 1))
normal = torch.zeros((len(loss_neg), 1))
y = torch.cat((anomalous, normal), 0)
scores = torch.cat((loss_pos, loss_neg), 0)
fpr, tpr, thresholds = metrics.roc_curve(y.cpu(), scores.cpu())
auc = metrics.auc(fpr, tpr)
auroc[seed] = auc
print ('AUC', auc)

# random seed 100

In [ ]:
gen.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/100/Leukemia1000epochs2020-03-15-19-13-28G'))
enc.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/100/Leukemia1000epochs2020-03-15-19-13-28E'))
dis_xz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/100/Leukemia1000epochs2020-03-15-19-13-28D_xz'))
dis_xx.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/100/Leukemia1000epochs2020-03-15-19-13-28D_xx'))
dis_zz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/100/Leukemia1000epochs2020-03-15-19-13-28D_zz'))

In [ ]:
dis_xz.to(device)
dis_xx.to(device)
dis_zz.to(device)
enc.to(device)
gen.to(device);

In [ ]:
dis_xz.eval()
dis_xx.eval()
dis_zz.eval()
enc.eval()
gen.eval()

loss_neg = torch.zeros((5,1)).cuda()
loss_pos = torch.zeros((15,1)).cuda()
c_neg = c_pos = 0
c_neg_val = c_pos_val = cntr = 0
for step, (images, labels, path) in enumerate(test_loader, 0):
    if '0.tif' in str(path):
        if c_neg_val < 5:
            images = images.view(-1, 3, 220, 220)
            c_neg_val += 1
            cntr += 1
        else:
            break
    elif '1.tif' in str(path):
        if c_pos_val < 15:
            images = images.view(-1, 3, 220, 220)
            c_pos_val += 1
            cntr += 1
        else:
            break
    
    x_real_test = images.cuda()
    z_random = torch.randn(images.shape[0], latent_size, 1, 1).cuda()
    z_gen = enc(x_real_test)
    x_gen = gen(z_random)
    rec_x = gen(z_gen)
    rec_z = enc(x_gen)
    
    l_gen, _ = dis_xz(x_real_test, z_gen)
    l_enc, _ = dis_xz(x_gen, z_random)
    
    x_logit_real, inter_layer_inp = dis_xx(x_real_test, x_real_test)
    x_logit_fake, inter_layer_rct = dis_xx(x_real_test, rec_x)
   
    fm = inter_layer_inp - inter_layer_rct
    feature_loss = torch.norm(fm, 1, keepdim=False) 
    feature_loss = feature_loss.squeeze()
    
    if '0.tif' in str(path):
        loss_neg[c_neg] = feature_loss.detach()
        c_neg += 1
    else:
        loss_pos[c_pos] = feature_loss.detach()
        c_pos += 1

    if cntr == 20:
        break
mean_neg = torch.mean(loss_neg)
mean_pos = torch.mean(loss_pos)
total_mean = (mean_neg + mean_pos) / 2
print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))
print ('threshold: %0.4f' %(total_mean))

In [ ]:
lam = 0.1
seed = 100
loss_neg = torch.zeros((100,1)).cuda()
loss_pos = torch.zeros((130,1)).cuda()
c_loss = c_neg = c_pos = 0
for step, (images, labels, path) in enumerate(test_loader, 0):
    images = images.view(-1, 3, 220, 220)
    dis_xz.eval()
    dis_xx.eval()
    dis_zz.eval()
    enc.eval()
    gen.eval()
    
    x_real_test = images.cuda()
    z_random = torch.randn(images.shape[0], latent_size, 1, 1).cuda()
    z_gen = enc(x_real_test)
    x_gen = gen(z_random)
    rec_x = gen(z_gen)
    rec_z = enc(x_gen)
    
    l_gen, _ = dis_xz(x_real_test, z_gen)
    l_enc, _ = dis_xz(x_gen, z_random)
    
    x_logit_real, inter_layer_inp = dis_xx(x_real_test, x_real_test)
    x_logit_fake, inter_layer_rct = dis_xx(x_real_test, rec_x)
   
    fm = inter_layer_inp - inter_layer_rct
    feature_loss = torch.norm(fm, 1, keepdim=False) 
    feature_loss = feature_loss.squeeze()
    
    if '0.tif' in str(path):
        loss_neg[c_neg] = feature_loss.detach()
        c_neg += 1
        c_loss += 1
    else:
        loss_pos[c_pos] = feature_loss.detach()
        c_pos += 1
        c_loss += 1
print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))

In [ ]:
FP = TP = []
neg_pre_wrong = 0
for i in range(len(loss_neg)):
    if loss_neg[i] > 1900:
        neg_pre_wrong += 1

pos_pre_wrong = 0
for i in range(len(loss_pos)):
    if loss_pos[i] <= 1900:
        pos_pre_wrong += 1
print ("number of normal samples missclassified: %d, number of anomalous samples missclassified: %d" 
       %(neg_pre_wrong, pos_pre_wrong))
tp = (len(loss_pos) - pos_pre_wrong)
fn = pos_pre_wrong
fp = neg_pre_wrong
tn = len(loss_neg) - neg_pre_wrong
precision = tp / (tp + fp)
## recall / sensitivity / True Positive Rate
recall = tp / (tp + fn)
## False Positive Rate / 1 - Specificity
fp_rate = fp / (fp + tn)
specificity = tn / (tn + fp)
f1 = 2 * ((precision * recall)/(precision + recall))
accuracy = (tp + tn) / (tp + tn + fp + fn)
acc_lam[seed] = accuracy
print ("tp: %d, fp: %d, fn: %d, tn: %d" %(tp, fp, fn, tn))
print ("precision: %.5f, recall: %.5f, specificity: %.5f, f1: %.5f, fp_rate: %.5f, accuracy: %.5f" 
       %(precision, recall, specificity, f1, fp_rate, accuracy))
anomalous = torch.ones((len(loss_pos), 1))
normal = torch.zeros((len(loss_neg), 1))
y = torch.cat((anomalous, normal), 0)
scores = torch.cat((loss_pos, loss_neg), 0)
fpr, tpr, thresholds = metrics.roc_curve(y.cpu(), scores.cpu())
auc = metrics.auc(fpr, tpr)
auroc[seed] = auc
print ('AUC', auc)

In [ ]:
# with new loos and betha = 0.01
sen = [0.2154, 1.0000, 1.0000]
spe = [0.8333, 0.7200, 0.7000]
f1 = [0.3436, 0.9028, 0.8966]
acc = [0.3313, 0.8783, 0.8696]
auc = [0.6692, 0.8416, 0.7868]

sen_mean = np.mean(sen)
sen_std = np.std(sen)

spe_mean = np.mean(spe)
spe_std = np.std(spe)

f1_mean = np.mean(f1)
f1_std = np.std(f1)

acc_mean = np.mean(acc)
acc_std = np.std(acc)

auc_mean = np.mean(auc)
auc_std = np.std(auc)


print (sen_mean, sen_std)
print (spe_mean, spe_std)
print (f1_mean, f1_std)
print (acc_mean, acc_std)
print (auc_mean, auc_std)

# precision: 0.84848, recall: 0.21538, specificity: 0.83333, f1: 0.34356, fp_rate: 0.16667, accuracy: 0.33125
# AUC 0.6692307692307693

In [ ]:
print (acc_lam)
print (auroc)

In [ ]:
with open('./models/Leukemia/RandomSeedTest/acc_seed.txt', 'w', newline='\n')as f:
    f.write(str(acc_lam))
with open('./models/Leukemia/RandomSeedTest/auroc_seed.txt', 'w', newline='\n')as f:
    f.write(str(auroc))

In [ ]:
print ('Model 1')
gen.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/0/Leukemia1000epochs2020-03-15-12-25-32G'))
enc.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/0/Leukemia1000epochs2020-03-15-12-25-32E'))
dis_xz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/0/Leukemia1000epochs2020-03-15-12-25-32D_xz'))
dis_xx.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/0/Leukemia1000epochs2020-03-15-12-25-32D_xx'))
dis_zz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/0/Leukemia1000epochs2020-03-15-12-25-32D_zz'))

dis_xz.to(device)
dis_xx.to(device)
dis_zz.to(device)
enc.to(device)
gen.to(device);

start_time = time.time()
lam = 0.1
seed = 0
loss_neg = torch.zeros((100,1)).cuda()
loss_pos = torch.zeros((130,1)).cuda()
c_loss = c_neg = c_pos = 0
for step, (images, labels, path) in enumerate(test_loader, 0):
    images = images.view(-1, 3, 220, 220)
    dis_xz.eval()
    dis_xx.eval()
    dis_zz.eval()
    enc.eval()
    gen.eval()
    
    x_real_test = images.cuda()
    z_random = torch.randn(images.shape[0], latent_size, 1, 1).cuda()
    z_gen = enc(x_real_test)
    x_gen = gen(z_random)
    rec_x = gen(z_gen)
    rec_z = enc(x_gen)
    
    l_gen, _ = dis_xz(x_real_test, z_gen)
    l_enc, _ = dis_xz(x_gen, z_random)
    
    x_logit_real, inter_layer_inp = dis_xx(x_real_test, x_real_test)
    x_logit_fake, inter_layer_rct = dis_xx(x_real_test, rec_x)
   
    fm = inter_layer_inp - inter_layer_rct
    feature_loss = torch.norm(fm, 1, keepdim=False) 
    feature_loss = feature_loss.squeeze()
    
    if '0.tif' in str(path):
        loss_neg[c_neg] = feature_loss.detach()
        c_neg += 1
        c_loss += 1
    else:
        loss_pos[c_pos] = feature_loss.detach()
        c_pos += 1
        c_loss += 1
# print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
# print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))

end_time = time.time() - start_time
print ('time:', end_time)

x1 = loss_neg.cpu().numpy()
x2 = loss_pos.cpu().numpy()


FP = TP = []
neg_pre_wrong = 0
for i in range(len(loss_neg)):
    if loss_neg[i] > 4100:
        neg_pre_wrong += 1

pos_pre_wrong = 0
for i in range(len(loss_pos)):
    if loss_pos[i] <= 4100:
        pos_pre_wrong += 1
print ("number of normal samples missclassified: %d, number of anomalous samples missclassified: %d" 
       %(neg_pre_wrong, pos_pre_wrong))
tp = (len(loss_pos) - pos_pre_wrong)
fn = pos_pre_wrong
fp = neg_pre_wrong
tn = len(loss_neg) - neg_pre_wrong
precision = tp / (tp + fp)
## recall / sensitivity / True Positive Rate
recall = tp / (tp + fn)
## False Positive Rate / 1 - Specificity
fp_rate = fp / (fp + tn)
specificity = tn / (tn + fp)
f1 = 2 * ((precision * recall)/(precision + recall))
accuracy = (tp + tn) / (tp + tn + fp + fn)
acc_lam[seed] = accuracy
print ("tp: %d, fp: %d, fn: %d, tn: %d" %(tp, fp, fn, tn))
print ("precision: %.5f, recall: %.5f, specificity: %.5f, f1: %.5f, fp_rate: %.5f, accuracy: %.5f" 
       %(precision, recall, specificity, f1, fp_rate, accuracy))
anomalous = torch.ones((len(loss_pos), 1))
normal = torch.zeros((len(loss_neg), 1))
y = torch.cat((anomalous, normal), 0)
scores = torch.cat((loss_pos, loss_neg), 0)
fpr_model1, tpr_model1, thresholds = metrics.roc_curve(y.cpu(), scores.cpu())
auc_model1 = metrics.auc(fpr_model1, tpr_model1)
auroc[seed] = auc_model1
print ('AUC', auc_model1)


print ('Model 2')
gen.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/10/Leukemia1000epochs2020-03-15-17-59-22G'))
enc.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/10/Leukemia1000epochs2020-03-15-17-59-22E'))
dis_xz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/10/Leukemia1000epochs2020-03-15-17-59-22D_xz'))
dis_xx.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/10/Leukemia1000epochs2020-03-15-17-59-22D_xx'))
dis_zz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/10/Leukemia1000epochs2020-03-15-17-59-22D_zz'))

dis_xz.to(device)
dis_xx.to(device)
dis_zz.to(device)
enc.to(device)
gen.to(device);

start_time = time.time()
lam = 0.1
seed = 10
loss_neg = torch.zeros((100,1)).cuda()
loss_pos = torch.zeros((130,1)).cuda()
c_loss = c_neg = c_pos = 0
for step, (images, labels, path) in enumerate(test_loader, 0):
    images = images.view(-1, 3, 220, 220)
    dis_xz.eval()
    dis_xx.eval()
    dis_zz.eval()
    enc.eval()
    gen.eval()
    
    x_real_test = images.cuda()
    z_random = torch.randn(images.shape[0], latent_size, 1, 1).cuda()
    z_gen = enc(x_real_test)
    x_gen = gen(z_random)
    rec_x = gen(z_gen)
    rec_z = enc(x_gen)
    
    l_gen, _ = dis_xz(x_real_test, z_gen)
    l_enc, _ = dis_xz(x_gen, z_random)
    
    x_logit_real, inter_layer_inp = dis_xx(x_real_test, x_real_test)
    x_logit_fake, inter_layer_rct = dis_xx(x_real_test, rec_x)
   
    fm = inter_layer_inp - inter_layer_rct
    feature_loss = torch.norm(fm, 1, keepdim=False) 
    feature_loss = feature_loss.squeeze()
    
    if '0.tif' in str(path):
        loss_neg[c_neg] = feature_loss.detach()
        c_neg += 1
        c_loss += 1
    else:
        loss_pos[c_pos] = feature_loss.detach()
        c_pos += 1
        c_loss += 1
# print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
# print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))

end_time = time.time() - start_time
print ('time:', end_time)

x1 = loss_neg.cpu().numpy()
x2 = loss_pos.cpu().numpy()


FP = TP = []
neg_pre_wrong = 0
for i in range(len(loss_neg)):
    if loss_neg[i] > 1200:
        neg_pre_wrong += 1

pos_pre_wrong = 0
for i in range(len(loss_pos)):
    if loss_pos[i] <= 1200:
        pos_pre_wrong += 1
print ("number of normal samples missclassified: %d, number of anomalous samples missclassified: %d" 
       %(neg_pre_wrong, pos_pre_wrong))
tp = (len(loss_pos) - pos_pre_wrong)
fn = pos_pre_wrong
fp = neg_pre_wrong
tn = len(loss_neg) - neg_pre_wrong
precision = tp / (tp + fp)
## recall / sensitivity / True Positive Rate
recall = tp / (tp + fn)
## False Positive Rate / 1 - Specificity
fp_rate = fp / (fp + tn)
specificity = tn / (tn + fp)
f1 = 2 * ((precision * recall)/(precision + recall))
accuracy = (tp + tn) / (tp + tn + fp + fn)
acc_lam[seed] = accuracy
print ("tp: %d, fp: %d, fn: %d, tn: %d" %(tp, fp, fn, tn))
print ("precision: %.5f, recall: %.5f, specificity: %.5f, f1: %.5f, fp_rate: %.5f, accuracy: %.5f" 
       %(precision, recall, specificity, f1, fp_rate, accuracy))
anomalous = torch.ones((len(loss_pos), 1))
normal = torch.zeros((len(loss_neg), 1))
y = torch.cat((anomalous, normal), 0)
scores = torch.cat((loss_pos, loss_neg), 0)
fpr_model2, tpr_model2, thresholds = metrics.roc_curve(y.cpu(), scores.cpu())
auc_model2 = metrics.auc(fpr_model2, tpr_model2)
auroc[seed] = auc_model2
print ('AUC', auc_model2)


print ('Model 3')
gen.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/100/Leukemia1000epochs2020-03-15-19-13-28G'))
enc.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/100/Leukemia1000epochs2020-03-15-19-13-28E'))
dis_xz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/100/Leukemia1000epochs2020-03-15-19-13-28D_xz'))
dis_xx.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/100/Leukemia1000epochs2020-03-15-19-13-28D_xx'))
dis_zz.load_state_dict(torch.load('./models/Leukemia/RandomSeedTest/100/Leukemia1000epochs2020-03-15-19-13-28D_zz'))

dis_xz.to(device)
dis_xx.to(device)
dis_zz.to(device)
enc.to(device)
gen.to(device);

start_time = time.time()
lam = 0.1
seed = 100
loss_neg = torch.zeros((100,1)).cuda()
loss_pos = torch.zeros((130,1)).cuda()
c_loss = c_neg = c_pos = 0
for step, (images, labels, path) in enumerate(test_loader, 0):
    images = images.view(-1, 3, 220, 220)
    dis_xz.eval()
    dis_xx.eval()
    dis_zz.eval()
    enc.eval()
    gen.eval()
    
    x_real_test = images.cuda()
    z_random = torch.randn(images.shape[0], latent_size, 1, 1).cuda()
    z_gen = enc(x_real_test)
    x_gen = gen(z_random)
    rec_x = gen(z_gen)
    rec_z = enc(x_gen)
    
    l_gen, _ = dis_xz(x_real_test, z_gen)
    l_enc, _ = dis_xz(x_gen, z_random)
    
    x_logit_real, inter_layer_inp = dis_xx(x_real_test, x_real_test)
    x_logit_fake, inter_layer_rct = dis_xx(x_real_test, rec_x)
   
    fm = inter_layer_inp - inter_layer_rct
    feature_loss = torch.norm(fm, 1, keepdim=False) 
    feature_loss = feature_loss.squeeze()
    
    if '0.tif' in str(path):
        loss_neg[c_neg] = feature_loss.detach()
        c_neg += 1
        c_loss += 1
    else:
        loss_pos[c_pos] = feature_loss.detach()
        c_pos += 1
        c_loss += 1
# print ('mean negative: %0.4f, std negative: %0.4f' %(torch.mean(loss_neg), torch.std(loss_neg)))
# print ('mean positive: %0.4f, std positive: %0.4f' %(torch.mean(loss_pos), torch.std(loss_pos)))

end_time = time.time() - start_time
print ('time:', end_time)

x1 = loss_neg.cpu().numpy()
x2 = loss_pos.cpu().numpy()


FP = TP = []
neg_pre_wrong = 0
for i in range(len(loss_neg)):
    if loss_neg[i] > 2720:
        neg_pre_wrong += 1

pos_pre_wrong = 0
for i in range(len(loss_pos)):
    if loss_pos[i] <= 2720:
        pos_pre_wrong += 1
print ("number of normal samples missclassified: %d, number of anomalous samples missclassified: %d" 
       %(neg_pre_wrong, pos_pre_wrong))
tp = (len(loss_pos) - pos_pre_wrong)
fn = pos_pre_wrong
fp = neg_pre_wrong
tn = len(loss_neg) - neg_pre_wrong
precision = tp / (tp + fp)
## recall / sensitivity / True Positive Rate
recall = tp / (tp + fn)
## False Positive Rate / 1 - Specificity
fp_rate = fp / (fp + tn)
specificity = tn / (tn + fp)
f1 = 2 * ((precision * recall)/(precision + recall))
accuracy = (tp + tn) / (tp + tn + fp + fn)
acc_lam[seed] = accuracy
print ("tp: %d, fp: %d, fn: %d, tn: %d" %(tp, fp, fn, tn))
print ("precision: %.5f, recall: %.5f, specificity: %.5f, f1: %.5f, fp_rate: %.5f, accuracy: %.5f" 
       %(precision, recall, specificity, f1, fp_rate, accuracy))
anomalous = torch.ones((len(loss_pos), 1))
normal = torch.zeros((len(loss_neg), 1))
y = torch.cat((anomalous, normal), 0)
scores = torch.cat((loss_pos, loss_neg), 0)
fpr_model3, tpr_model3, thresholds = metrics.roc_curve(y.cpu(), scores.cpu())
auc_model3 = metrics.auc(fpr_model3, tpr_model3)
auroc[seed] = auc_model3
print ('AUC', auc_model3)


plt.figure()
plt.plot(fpr_model1, tpr_model1, color='darkorange', label='Model 1(area = %0.3f)' % auc_model1)
plt.plot(fpr_model2, tpr_model2, color='green', label='Model 2(area = %0.3f)' % auc_model2)
plt.plot(fpr_model3, tpr_model3, color='red', label='Model 3(area = %0.3f)' % auc_model3)
plt.plot([0.0, 1.0], color='navy', linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([-0.01, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show();

In [ ]:
lists = sorted(acc_lam.items())
x, y = zip(*lists)
plt.plot(x, y, color='darkorange')
plt.xlabel('Random Initialization')
plt.ylabel('Accuracy')
plt.title('The accuracy of ALAD model for different random initialization')
plt.show()

In [ ]:
auc = [0.7390, 0.8857, 0.7950]

auc_mean = np.mean(auc)
auc_std = np.std(auc)

print (auc_mean, auc_std)

In [ ]:
sen = [0.9692, 0.9385, 0.9307]
spe = [0.7000, 0.8300, 0.7200]
f1 = [0.8811, 0.9071, 0.8674]
acc = [0.8522, 0.8913, 0.8391]
auc = [0.7480, 0.8634, 0.7850]

sen_mean = np.mean(sen)
sen_std = np.std(sen)

spe_mean = np.mean(spe)
spe_std = np.std(spe)

f1_mean = np.mean(f1)
f1_std = np.std(f1)

acc_mean = np.mean(acc)
acc_std = np.std(acc)

auc_mean = np.mean(auc)
auc_std = np.std(auc)


print (sen_mean, sen_std)
print (spe_mean, spe_std)
print (f1_mean, f1_std)
print (acc_mean, acc_std)
print (auc_mean, auc_std)